# Tally Arithmetic
This notebook introduce tally arithmetic, including how to add, subtrate and multiple tallies.
Note that it is assumed that tallies are independent, since covariance is not obtained.

In [19]:
import openmc
import glob
import numpy as np
import matplotlib.pyplot as plt

## Modeling
Materials, geometry and settings of a pin-cell problem.

In [20]:
# Materials

fuel = openmc.Material(name='fuel', temperature=273.0)
fuel.add_nuclide('U235', 3.7503e-4)
fuel.add_nuclide('U238', 2.2625e-2)
fuel.add_nuclide('O16', 4.6007e-2)
fuel.set_density('g/cm3', 10.31341)

clad = openmc.Material(name='clad', temperature=273.0)
clad.add_nuclide('Zr90', 7.2758e-3)
clad.set_density('g/cm3', 6.55)

water = openmc.Material(name='water', temperature=273.0)
water.add_nuclide('H1', 4.9457e-2)
water.add_nuclide('O16', 2.4732e-2)
water.add_nuclide('B10', 8.0042e-6)
water.set_density('g/cm3', 0.470582)

materails = openmc.Materials([fuel, clad, water])
materails.export_to_xml()

In [21]:
# Geometry

fuel_radius = 0.39328
clad_radius = 0.45720
half_pitch = 0.63
pin_half_length = 100.

fuel_outer_surface = openmc.ZCylinder(r=fuel_radius)
clad_outer_surface = openmc.ZCylinder(r=clad_radius)

xmin = openmc.XPlane(x0=-half_pitch, boundary_type='reflective')
xmax = openmc.XPlane(x0=+half_pitch, boundary_type='reflective')
ymin = openmc.YPlane(y0=-half_pitch, boundary_type='reflective')
ymax = openmc.YPlane(y0=+half_pitch, boundary_type='reflective')
zmin = openmc.ZPlane(z0=-pin_half_length, boundary_type='vacuum')
zmax = openmc.ZPlane(z0=+pin_half_length, boundary_type='vacuum')

fuel_cell = openmc.Cell(name='fuel', fill=fuel, region=-fuel_outer_surface)
clad_cell = openmc.Cell(name='clad', fill=clad, region=+fuel_outer_surface & -clad_outer_surface)
moderator_cell = openmc.Cell(name='moderator', fill=water, region=+clad_outer_surface)

pin_cell_universe = openmc.Universe(
    name='pin cell universe', 
    cells=[fuel_cell, clad_cell, moderator_cell]
)
root_cell = openmc.Cell(
    name='root cell', 
    fill=pin_cell_universe, 
    region=+xmin & -xmax & +ymin & -ymax & +zmin & -zmax
)
root_universe = openmc.Universe(
    name='root universe',
    cells=[root_cell]
)

geometry = openmc.Geometry(root_universe)
geometry.export_to_xml()

In [22]:
# Setting
setting = openmc.Settings()
setting.batches = 20
setting.inactive = 5
setting.particles = 2500
setting.output = {'tallies': True}

# Initial Source
bounds = [-half_pitch, -half_pitch, -pin_half_length, half_pitch, half_pitch, pin_half_length]
uniform_space = openmc.stats.Box(bounds[:3], bounds[3:], only_fissionable=True)
setting.source = openmc.Source(space=uniform_space)

setting.export_to_xml()

Plot the geometry and run OpenMC.

In [23]:
# Plot

plot = openmc.Plot(name='geometry plot')
plot.filename = 'pin-cell-geometry'
plot.pixels = [500, 500]
plot.origin = [0, 0, 0]
plot.width = [1.26, 1.26]
plot.color_by = 'material'

# openmc.plot_inline(plot)
# .ppm plot has been created in directory, but it can not be show inline and assert an error below. Never mind.

### Tally
Note that we will use 4-factor formula to calculate the k-eigenvalue (infinite).

In [24]:
tallies = openmc.Tallies()
energy_filter = openmc.EnergyFilter([0., 0.625, 20.0e6])

# Flux Tally
tally = openmc.Tally(name='flux')
tally.filters = [openmc.CellFilter([fuel_cell, moderator_cell]), energy_filter]
tally.scores = ['flux']
tallies.append(tally)

# Fuel Reaction Rate Tally
tally = openmc.Tally(name='fuel reaction')
tally.filters = [openmc.CellFilter(fuel_cell), energy_filter]
tally.scores = ['nu-fission', 'scatter']
tally.nuclides = ['U235', 'U238']
tallies.append(tally)

# Moderator Reaction Rate
tally = openmc.Tally(name='moderator reaction')
tally.filters = [openmc.CellFilter(moderator_cell), energy_filter]
tally.scores = ['absorption', 'total']
tally.nuclides = ['H1', 'O16']
tallies.append(tally)

# Fuel & Moderator Reaction Rate
tally = openmc.Tally(name='total reaction')
tally.filters = [openmc.CellFilter([fuel_cell, moderator_cell]), energy_filter]
tally.scores = ['absorption', 'scatter']
tally.nuclides = ['H1', 'U235']
tallies.append(tally)

# K-eigenvalue - Definition
# Total Leakage
mesh = openmc.RegularMesh()
mesh.dimension = [1, 1, 1]
mesh.lower_left = [-half_pitch, -half_pitch, -pin_half_length]
mesh.width = [2 * half_pitch, 2 * half_pitch, 2 * pin_half_length]
mesh_filter = openmc.MeshSurfaceFilter(mesh)

tally = openmc.Tally(name='total leak')
tally.filters = [mesh_filter]
tally.scores = ['current']
tallies.append(tally)

# Thermal Leakage
tally = openmc.Tally(name='thermal leak')
tally.filters = [mesh_filter, openmc.EnergyFilter([0., 0.625])]
tally.scores = ['current']
tallies.append(tally)

# Fast Leakage
tally = openmc.Tally(name='fast leak')
tally.filters = [mesh_filter, openmc.EnergyFilter([0.625, 20.0e6])]
tally.scores = ['current']
tallies.append(tally)

# Fission and Absorption Rate
tally = openmc.Tally(name='fission')
tally.scores = ['nu-fission']
tallies.append(tally)
tally = openmc.Tally(name='absorption')
tally.scores = ['absorption']
tallies.append(tally)

# k-eigenvalue - 6-factor formula
# Fast Fission Factor \epsilon
tally = openmc.Tally(name='thermal fission')
tally.filters = [openmc.EnergyFilter([0., 0.625])]
tally.scores = ['nu-fission']
tallies.append(tally)

# Resonance Capture Factor \p
tally = openmc.Tally(name='thermal absorption')
tally.filters = [openmc.EnergyFilter([0., 0.625])]
tally.scores = ['absorption']
tallies.append(tally)

# Thermal Nuetron Utilization Factor \f
tally = openmc.Tally(name='fuel thermal absorption')
tally.filters = [openmc.EnergyFilter((0., 0.625)), openmc.CellFilter(fuel_cell)]
tally.scores = ['absorption']
tallies.append(tally)

tallies.export_to_xml()

/home/zikangli/.local/lib/python3.8/site-packages/openmc/mixin.py:67: IDWarning: Another Filter instance already exists with id=2.
  warn(msg, IDWarning)
/home/zikangli/.local/lib/python3.8/site-packages/openmc/mixin.py:67: IDWarning: Another Filter instance already exists with id=7.
  warn(msg, IDWarning)
/home/zikangli/.local/lib/python3.8/site-packages/openmc/mixin.py:67: IDWarning: Another Filter instance already exists with id=3.
  warn(msg, IDWarning)


In [25]:
# Run
openmc.run()

                                %%%%%%%%%%%%%%%
                           %%%%%%%%%%%%%%%%%%%%%%%%
                        %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                      %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                    %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                   %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                                    %%%%%%%%%%%%%%%%%%%%%%%%
                                     %%%%%%%%%%%%%%%%%%%%%%%%
                 ###############      %%%%%%%%%%%%%%%%%%%%%%%%
                ##################     %%%%%%%%%%%%%%%%%%%%%%%
                ###################     %%%%%%%%%%%%%%%%%%%%%%%
                ####################     %%%%%%%%%%%%%%%%%%%%%%
                #####################     %%%%%%%%%%%%%%%%%%%%%
                ######################     %%%%%%%%%%%%%%%%%%%%
                #######################     %%%%%%%%%%%%%%%%%%
                 #######################     %%%%%%%%%%%%%%%%%
                 #####################

## Tally Data Processing and Analysis
Read state point file.

In [26]:
statepoints = glob.glob('statepoint.*.h5')
sp = openmc.StatePoint(statepoints[-1])

### Calculate k-eigenvalue using definition
The k-eigenvalue is defined as given
$$
k_{eff} = \frac{<\nu \Sigma _f \phi >}{<\Sigma _a \phi> + <L>}
$$

Extract relative data from tallies and calculate it with tally arithmetic as below.

In [27]:
fission = sp.get_tally(name='fission')
absorption = sp.get_tally(name='absorption')
leakage = sp.get_tally(name='total leak')
leakage = leakage.summation(filter_type=openmc.MeshSurfaceFilter, remove_filter=True)

keff = fission / (absorption + leakage)
keff_def = keff.get_pandas_dataframe()
keff_def

,nuclide,score,mean,std. dev.
0,total,(nu-fission / (absorption + current)),1.025129,0.007841


### Calculate k-eigenvalue using 6-factor formula
Then 6-factor formula is given as
$$
k_{eff} = \epsilon pf \eta \Lambda _s \Lambda _f
$$
whereas the fast fission factor is
$$
\epsilon = \frac{<\nu \Sigma _f \phi >}{{<\nu \Sigma _f \phi >}_{thermal}}
$$

In [28]:
fission_thermal = sp.get_tally(name='thermal fission')

epsilon = fission / fission_thermal
epsilon.get_pandas_dataframe()

,energy low [eV],energy high [eV],nuclide,score,mean,std. dev.
0,0.0,0.625,total,(nu-fission / nu-fission),1.297072,0.012905


Resonance Escape Possibility
$$
p = \frac{<\Sigma _a \phi >_{thermal} + <L>_{thermal}}{<\Sigma _a \phi > + <L>_{thermal}}
$$
there are 2 $<L>_{thermal}$ because p does not consider the effect of leakage.

In [29]:
absorption_thermal = sp.get_tally(name='thermal absorption')
leakage_thermal = sp.get_tally(name='thermal leak')
leakage_thermal = leakage_thermal.summation(filter_type=openmc.MeshSurfaceFilter, remove_filter=True)

p = (absorption_thermal + leakage_thermal) / (absorption + leakage_thermal)
p.get_pandas_dataframe()

,energy low [eV],energy high [eV],nuclide,score,mean,std. dev.
0,0.0,0.625,total,((absorption + current) / (absorption + current)),0.591539,0.005419


Thermal Nuetron Utilization Factor
$$
f = \frac{{<\Sigma _f \phi >}_{fuel, thermal}}{{<\Sigma _f \phi >}_{thermal}}
$$

In [30]:
fuel_thermal_absorption = sp.get_tally(name='fuel thermal absorption')

f = fuel_thermal_absorption / absorption_thermal
f.get_pandas_dataframe()

,energy low [eV],energy high [eV],cell,nuclide,score,mean,std. dev.
0,0.0,0.625,13,total,(absorption / absorption),0.825336,0.009179


Effective Nuetron Production Factor
$$
\eta = \frac{{<\nu \Sigma _f \phi >}_{thermal}}{{<\Sigma _a \phi >_{thermal}}}
$$

In [31]:
eta = fission_thermal / fuel_thermal_absorption
eta.get_pandas_dataframe()

,energy low [eV],energy high [eV],cell,nuclide,score,mean,std. dev.
0,0.0,0.625,13,total,(nu-fission / absorption),1.660728,0.018821


Fast Non-Leakage Probability
$$
\Lambda _s = \frac{<\Sigma _a \phi > + {<L> _{thermal}}}{<\Sigma _a \phi > + <L> }
$$

In [32]:
fast_non_leak = (absorption + leakage_thermal) / (absorption + leakage)
fast_non_leak.get_pandas_dataframe()

,energy low [eV],energy high [eV],nuclide,score,mean,std. dev.
0,0.0,0.625,total,((absorption + current) / (absorption + current)),0.978009,0.006808


Thermal Non-Leakage Probability
$$
\Lambda _f = \frac{{<\Sigma _a \phi >}_{thermal}}{{<\Sigma _a \phi >}_{thermal} + {<L>}_{thermal} }
$$

In [33]:
thermal_non_leak = absorption_thermal / (absorption_thermal + leakage_thermal)
thermal_non_leak.get_pandas_dataframe()

,energy low [eV],energy high [eV],nuclide,score,mean,std. dev.
0,0.0,0.625,total,(absorption / (absorption + current)),0.996689,0.010892


Finally, we get the k-eigenvalue with 6-factor formula.

In [34]:
keff_formula = epsilon * p * f * eta * fast_non_leak * thermal_non_leak
keff_formula.get_pandas_dataframe()

,energy low [eV],energy high [eV],cell,nuclide,score,mean,std. dev.
0,0.0,0.625,13,total,((((((nu-fission / nu-fission) * ((absorption ...,1.025129,0.025171


To conclusion, we get the same k-eigenvalue through defination and 6-factor formula, while the deviation of formula is larger than of definition.

### Calculate Microscopic Multi-group Cross Sections
$$
\sigma _x = \frac{<R_x>}{<\phi >}
$$

In [35]:
reaction_rate = sp.get_tally(name='total reaction')
flux = sp.get_tally(name='flux')
flux = flux.get_slice(filters=[openmc.CellFilter], filter_bins=[(fuel_cell.id, )])
reaction_fuel = sp.get_tally(name='fuel reaction')
reaction_moderator = sp.get_tally(name='moderator reaction')

In [36]:
fuel_micro_xs = reaction_fuel / flux
fuel_micro_xs.get_pandas_dataframe()

,cell,energy low [eV],energy high [eV],nuclide,score,mean,std. dev.
0,13,0.000,6.250000e-01,(U235 / total),(nu-fission / flux),3.378729e-01,3.634886e-03
1,13,0.000,6.250000e-01,(U235 / total),(scatter / flux),5.535983e-03,5.604502e-05
2,13,0.000,6.250000e-01,(U238 / total),(nu-fission / flux),6.341334e-07,6.738183e-09
3,13,0.000,6.250000e-01,(U238 / total),(scatter / flux),2.098916e-01,2.123143e-03
4,13,0.625,2.000000e+07,(U235 / total),(nu-fission / flux),7.907702e-03,5.321264e-05
5,13,0.625,2.000000e+07,(U235 / total),(scatter / flux),3.401896e-03,1.789082e-05
6,13,0.625,2.000000e+07,(U238 / total),(nu-fission / flux),6.544572e-03,6.409336e-05
7,13,0.625,2.000000e+07,(U238 / total),(scatter / flux),2.291472e-01,1.175361e-03


We generated fission & scatter micro cross-sections above. However, if we want only fission cross-sections, we could extract them via the method below.

In [38]:
fission_xs = fuel_micro_xs.get_values(scores=['(nu-fission / flux)'])
fission_xs

array([[[3.37872856e-01],
        [6.34133416e-07]],

       [[7.90770198e-03],
        [6.54457158e-03]]])

which is same as

In [39]:
df_fuel = fuel_micro_xs.get_pandas_dataframe()
df_fission = df_fuel[df_fuel['score'] == '(nu-fission / flux)']
df_fission.head()

,cell,energy low [eV],energy high [eV],nuclide,score,mean,std. dev.
0,13,0.000,6.250000e-01,(U235 / total),(nu-fission / flux),3.378729e-01,3.634886e-03
2,13,0.000,6.250000e-01,(U238 / total),(nu-fission / flux),6.341334e-07,6.738183e-09
4,13,0.625,2.000000e+07,(U235 / total),(nu-fission / flux),7.907702e-03,5.321264e-05
6,13,0.625,2.000000e+07,(U238 / total),(nu-fission / flux),6.544572e-03,6.409336e-05


In [42]:
u235_scatter_xs = fuel_micro_xs.get_values(
    nuclides=['(U235 / total)'], 
    scores=['(scatter / flux)']
)
u238_scatter_xs = fuel_micro_xs.get_values(
    nuclides=['(U238 / total)'], 
    scores=['(scatter / flux)']
)
u235_scatter_xs

array([[[0.00553598]],

       [[0.0034019 ]]])

Then we generate fast scater micro cross-sections, which is identical to the cross-sections above.

In [44]:
fast_scatter_xs = fuel_micro_xs.get_values(
    filters=[openmc.EnergyFilter],
    filter_bins=[((0.625, 20.0e6), )],
    scores=['(scatter / flux)']
)
fast_scatter_xs

array([[[0.0034019 ],
        [0.22914717]]])